In [7]:
import pandas as pd
import numpy as np
import os

In [8]:
period_length = 252

In [9]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
tickers = df['Symbol'].tolist()

In [10]:
# Constants
DATE_REF_TICKER = "IBM"  # Ticker that serves as the date reference

# Load the IBM data as it's our date reference
ref_df = pd.read_csv(os.path.join("data", f"{DATE_REF_TICKER}.csv"), index_col="Date", parse_dates=True)
ref_df["Ticker"] = DATE_REF_TICKER
ref_df = ref_df[["Ticker", "Adj Close", "Volume"]]
ref_df.rename(columns={"Adj Close": "Price"}, inplace=True)

# Use the ref_df dataframe to reindex other stock dataframes to match its date range
reindexer = pd.DataFrame(index=ref_df.index)

# Create a container to store the dataframes
all_data = [ref_df]

# Load the Fed rate data and reindex it to match the date reference
fund_rates = pd.read_csv(os.path.join("data", "fed_funds_rate.csv"), index_col="Date", parse_dates=True)
fund_rates = reindexer.merge(fund_rates, left_index=True, right_index=True, how='left')
fund_rates["Ticker"] = "FED"
fund_rates.rename(columns={"Value": "Price"}, inplace=True)
fund_rates["Volume"] = 0
fund_rates["Price"] = (1 + fund_rates["Price"] / 100) ** (1 / 252) - 1
all_data.append(fund_rates)

# Get the list of tickers
valid_tickers = tickers.copy()

for ticker in tickers:
    # Avoid double counting
    if ticker == DATE_REF_TICKER:
        continue

    try:
        df = pd.read_csv(os.path.join("data", f"{ticker}.csv"), index_col="Date", parse_dates=True)
        df["Ticker"] = ticker
        df = df[["Ticker", "Adj Close", "Volume"]]
        df.rename(columns={"Adj Close": "Price"}, inplace=True)

        # Skip tickers with all NaN values in the "Price" column
        if df["Price"].isna().all():
            valid_tickers.remove(ticker)
            continue

        # Reindex the dataframe to match the date range in ref_df
        df = reindexer.merge(df, left_index=True, right_index=True, how='left')
        df["Ticker"].fillna(ticker, inplace=True)

        all_data.append(df)
    except FileNotFoundError:
        pass

# Concatenate the dataframes in all_data
merged_data = pd.concat(all_data)

# Assert the length of merged_data is as expected
assert len(merged_data) == (len(valid_tickers) + 1) * len(ref_df)

In [12]:
tickers = merged_data['Ticker'].unique()
tickers

array(['IBM', 'FED', 'MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM',
       'ADBE', 'ADP', 'AES', 'AFL', 'A', 'ABNB', 'APD', 'AKAM', 'ALK',
       'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO',
       'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT',
       'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA',
       'AAPL', 'AMAT', 'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'ATO',
       'ADSK', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BBWI',
       'BAX', 'BDX', 'WRB', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX',
       'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR',
       'BRO', 'BG', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH',
       'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE',
       'COR', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX',
       'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG',
       'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CA

In [23]:
list(tickers)

['IBM',
 'FED',
 'MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ATVI',
 'ADM',
 'ADBE',
 'ADP',
 'AES',
 'AFL',
 'A',
 'ABNB',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'WRB',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BG',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CDAY',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 

In [5]:
merged_data

,Ticker,Price,Volume
Date,,,
2000-01-03,IBM,61.718239,10823694.0
2000-01-04,IBM,59.623302,8606279.0
2000-01-05,IBM,61.718239,13318927.0
2000-01-06,IBM,60.654163,8338607.0
2000-01-07,IBM,60.388134,12402108.0
...,...,...,...
2023-06-23,ZTS,168.000519,2412100.0
2023-06-26,ZTS,167.172348,1729200.0
2023-06-27,ZTS,170.295486,1452300.0


In [6]:
# print merged data where ticker is FED
merged_data[merged_data["Ticker"] == "FED"]

,Ticker,Price,Volume
Date,,,
2000-01-03,FED,0.000210,0.0
2000-01-04,FED,0.000208,0.0
2000-01-05,FED,0.000209,0.0
2000-01-06,FED,0.000214,0.0
2000-01-07,FED,0.000217,0.0
...,...,...,...
2023-06-23,FED,NaN,0.0
2023-06-26,FED,NaN,0.0
2023-06-27,FED,NaN,0.0


In [7]:
merged_data.sort_index(inplace=True)

In [8]:
# count missing values in each column
merged_data.isna().sum()

Ticker         0
Price     326044
Volume    325804
dtype: int64

In [9]:
# Handle delisted companies
end_date = merged_data.index.max()
print(end_date)

2023-06-29 00:00:00


In [10]:
# show where ticker is BRK.B
merged_data[merged_data["Ticker"] == "BBY"]

,Ticker,Price,Volume
Date,,,
2000-01-03,BBY,15.775073,19442925.0
2000-01-04,BBY,15.672631,11525625.0
2000-01-05,BBY,15.570198,11111625.0
2000-01-06,BBY,15.433620,5968125.0
2000-01-07,BBY,15.911648,5736600.0
...,...,...,...
2023-06-23,BBY,78.124336,2807600.0
2023-06-26,BBY,80.365494,2448700.0
2023-06-27,BBY,81.856300,2568500.0


In [11]:
# Handle newly listed companies
start_date = merged_data.index.min()
for ticker in valid_tickers:
    ticker_data = merged_data[merged_data["Ticker"] == ticker]
    if ticker_data["Price"].head(1).isna().values[0]:
        first_non_nan_date = ticker_data["Price"].first_valid_index()
        condition = (merged_data["Ticker"] == ticker) & (merged_data.index < first_non_nan_date)
        merged_data.loc[condition, ["Price", "Volume"]] = -6666.6666

# Handle delisted companies
end_date = merged_data.index.max()
for ticker in valid_tickers:
    ticker_data = merged_data[merged_data["Ticker"] == ticker]
    if ticker_data["Price"].tail(1).isna().values[0]:
        last_non_nan_date = ticker_data["Price"].last_valid_index()
        condition = (merged_data["Ticker"] == ticker) & (merged_data.index > last_non_nan_date)
        merged_data.loc[condition, ["Price", "Volume"]] = -5555.5555

In [ ]:
merged_data.index.unique

In [14]:
# Remove tickers with over 1% missing values, per ticker -> we need the number of unique dates per ticker
threshold = 0.01 * len(merged_data.index.unique())
for ticker in valid_tickers:
    ticker_data = merged_data[merged_data["Ticker"] == ticker]
    if ticker_data["Price"].isna().sum() > threshold:
        merged_data = merged_data[merged_data["Ticker"] != ticker]
        # Remove ticker from valid_tickers
        valid_tickers.remove(ticker)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [21]:
assert (merged_data.index.unique().values == ref_df.index.unique().values).all()

array([ True,  True,  True, ...,  True,  True,  True])

In [22]:
# make sure that the data is sorted by date and ticker
merged_data.sort_values(by=["Date", "Ticker"], inplace=True)

# Forward fill NaN values, per ticker, make sure not to spill over to other tickers
for ticker in valid_tickers:
    ticker_data = merged_data[merged_data["Ticker"] == ticker].copy()
    ticker_data.ffill(inplace=True)
    merged_data.loc[merged_data["Ticker"] == ticker] = ticker_data

In [32]:
# replace the -6666.6666 and -5555.5555 values with NaN
merged_data.replace(-6666.6666, np.nan, inplace=True)
merged_data.replace(-5555.5555, np.nan, inplace=True)

In [24]:
# Ensure DataFrame is sorted
merged_data.sort_values(by=["Date", "Ticker"], inplace=True)

# Save the DataFrame
merged_data.to_csv(os.path.join("data", "data.csv"))

In [28]:
# assert that unique values in the "Ticker" column are the same as valid_tickers (sorted)
assert merged_data["Ticker"].unique().tolist().sort() == valid_tickers.sort()

from utils.data_object import InvestmentData
# Create the InvestmentData object
investment_data_obj = InvestmentData(merged_data)

# Save the InvestmentData object
investment_data_obj.save()

In [31]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
# only keep the rows where symbol is in the list of tickers
df = df[df['Symbol'].isin(tickers)]

  Symbol     Security             GICS Sector               GICS Sub-Industry  \
0    MMM           3M             Industrials        Industrial Conglomerates   
1    AOS  A. O. Smith             Industrials               Building Products   
2    ABT       Abbott             Health Care           Health Care Equipment   
3   ABBV       AbbVie             Health Care                 Pharmaceuticals   
4    ACN    Accenture  Information Technology  IT Consulting & Other Services   

     Headquarters Location  Date added      CIK      Founded  
0    Saint Paul, Minnesota  1957-03-04    66740         1902  
1     Milwaukee, Wisconsin  2017-07-26    91142         1916  
2  North Chicago, Illinois  1957-03-04     1800         1888  
3  North Chicago, Illinois  2012-12-31  1551152  2013 (1888)  
4          Dublin, Ireland  2011-07-06  1467373         1989  


In [30]:
#table

[    Symbol              Security             GICS Sector  \
 0      MMM                    3M             Industrials   
 1      AOS           A. O. Smith             Industrials   
 2      ABT                Abbott             Health Care   
 3     ABBV                AbbVie             Health Care   
 4      ACN             Accenture  Information Technology   
 ..     ...                   ...                     ...   
 498    YUM           Yum! Brands  Consumer Discretionary   
 499   ZBRA    Zebra Technologies  Information Technology   
 500    ZBH         Zimmer Biomet             Health Care   
 501   ZION  Zions Bancorporation              Financials   
 502    ZTS                Zoetis             Health Care   
 
                       GICS Sub-Industry    Headquarters Location  Date added  \
 0              Industrial Conglomerates    Saint Paul, Minnesota  1957-03-04   
 1                     Building Products     Milwaukee, Wisconsin  2017-07-26   
 2                 Heal

great, now let us reconsider the numpy data in the overall context of this repo. the repo is an investment game repo. there is a simulation at the core of this. the simulation is accessing, per time step in the pygame game, the data and calculating the return per period. what do you think is the most efficient way to store this? we need to make sure that it is a numpy array, right? how should the numpy array be designed? i propose 2 primary keys (tikcer and date) and then the values (price and volume). make sure that the np data type is appropriate. for ticker since it is a string, it might make sense to encode the string into numeric. the date should already be encoded as numeric, no? i am not too sure here. when creating the np array, make sure that each dtype is correct! then focus on what is the most appropriate way to store it and access it. i think a class wrapper might come in super handy here. does the wrapper need to be saved as pickle ? i guess right? the wrapper should include mapping dictionaries, to cobmine efficiency with human interpretability allowing me to keep trck and easily access the data later on. reason about all those points, then provide the corresponding code. note that this is stock price and volume data per day and ticker. also, every ticker has a corresponding company name (except for FED which translates to fed rate). conceptually reason abou t what is most appropriate to do. then, implement your solution accordingly

In [8]:
import os
import random
import pandas as pd
import numpy as np

from utils.data_object import InvestmentData

data_path = os.path.join(os.getcwd(), "data")

# Load the data object
data_obj = InvestmentData("investment_data")
print(data_obj.tickers[0])

def test_data_query():
    # Load tickers
    tickers = data_obj.tickers

    # Iterating randomly X times over tickers and dates
    NUM_TICKER_TESTS = 20
    NUM_DATE_TESTS = 1000
    TOLERANCE = 1e-6

    print(tickers)

    for _ in range(NUM_TICKER_TESTS):
        ticker = random.choice(tickers)

        # Load the CSV file using the updated path
        df = pd.read_csv(os.path.join(data_path, f"{ticker}.csv"), index_col="Date", parse_dates=True)

        print(df)

        for _ in range(NUM_DATE_TESTS):
            date = random.choice(df.index)

            # Querying the data object
            price_obj, volume_obj = data_obj.query(date, ticker)
            price_csv, volume_csv = df.loc[date, ["Adj Close", "Volume"]]

            # Handling NaNs
            if pd.isna(price_csv) or pd.isna(volume_obj):
                continue

            # Assertions with tolerance
            assert np.isclose(price_obj, price_csv,
                              atol=TOLERANCE), f"Expected {price_csv}, but got {price_obj} for {ticker} on {date}."
            assert np.isclose(volume_obj, volume_csv,
                              atol=TOLERANCE), f"Expected {volume_csv}, but got {volume_obj} for {ticker} on {date}."

if __name__ == "__main__":
    test_data_query()

TypeError: 'NoneType' object is not subscriptable